In [1]:
import findspark
findspark.init('/Users/wy/spark-1.6.0-bin-hadoop2.6')

import pyspark
sc = pyspark.SparkContext(appName="myAppName")

In [19]:
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from collections import Counter
from sklearn import tree
import random
import numpy as np
from sklearn import preprocessing
import datetime
from sklearn.ensemble import ExtraTreesClassifier
import pandas as pd
from pymongo import MongoClient

In [36]:
def evalution(y_test,predict):
    labelAndprediction = zip(list(y_test),list(predict))
    recallBase = 0
    recallUper = 0
    precisionBase = 0
    precisionUper = 0
    accuracy = 0

    for label,prediction in labelAndprediction:
        label = int(label)
        if label == 1 :
            recallBase += 1
            if prediction == 1 :
                recallUper += 1
    recall = float(recallUper)/float(recallBase)

    for label,prediction in labelAndprediction:
        prediction = int(prediction)
        if prediction == 1 :
            precisionBase += 1
            if label == 1 :
                precisionUper += 1
    precision = float(precisionUper)/float(precisionBase)

    for label,prediction in labelAndprediction:
        if label == prediction:
            accuracy += 1
    accuracy = float(accuracy)/float(len(y_test))
    f1score = (2*precision*recall)/(precision+recall)

    print "accuracy : %f " %(accuracy)
    print "recallBase : %f , recallUper : %f , recall : %f " %(recallBase, recallUper, recall)
    print "precisionBase : %f , precisionUper : %f , precision : %f " %(precisionBase, precisionUper, precision)
    print "f1score : %f " %(f1score)

In [5]:
iris = datasets.load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.25)
dataset = np.column_stack((X_train, y_train))

In [7]:
broadcastVar = sc.broadcast(dataset)

In [8]:
n_estimators = sc.parallelize(range(10),5)

In [9]:
n_estimators.collect()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [10]:
broadDataset = broadcastVar.value
n_estimators.map(lambda x : pallDecisiontree())

PythonRDD[1] at RDD at PythonRDD.scala:43

In [11]:
def pallDecisiontree(max_features="auto"):

    broadDataset = broadcastVar.value
    m,n = broadDataset.shape
    # 扣掉label
    n = n-1
    print m
    print n

    if max_features == 'auto':
        n_features = n
    elif max_features == 'sqrt':
        n_features = int(math.ceil(np.sqrt(n)))
    elif type(max_features).__name__=='float':
        n_features =  int(math.ceil(n*max_features))
    else:
        print "max_features error"
        
    sampleRindex = list(set(np.random.randint(m-1, size=m)))
    redRindex = list(set(range(m)).difference(set(sampleRindex)))
    featureRindex = list(set(random.sample(range(n), n_features)))
    datasetR = dataset[sampleRindex][:, featureRindex]
    labelR = dataset[sampleRindex][:, -1]
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(datasetR, labelR)     
    return clf

In [30]:
forest = n_estimators.map(lambda x : pallDecisiontree())

In [32]:
forestList = forest.collect()

In [42]:
def predictRF(forest, X_test):
    predict_2darray = []
    for treeIndex in range(len(forest)):
        predict_2darray.append(forest[treeIndex].predict(X_test))
    predict_2darray = np.array(predict_2darray)

    y_predict = []
    for a in range(len(X_test)):
        c = Counter(predict_2darray[:,a])
        constance = c.most_common(1)[0][0]
        p = c.most_common(1)[0][1]/float(len(predict_2darray[:,a]))
        y_predict.append(int(constance))
    return y_predict

In [43]:
y_predict = predictRF(forestList, X_test)

In [45]:
evalution(y_test,y_predict)

accuracy : 0.973684 
recallBase : 15.000000 , recallUper : 15.000000 , recall : 1.000000 
precisionBase : 16.000000 , precisionUper : 15.000000 , precision : 0.937500 
f1score : 0.967742 
